## Problem 1

In [3]:
def read_input(filename):
    f = open(f'../inputs/{filename}.txt', 'r')

    mtx = []
    while True:
        line = f.readline()
        if line == '':
            break
        mtx.append(list(line.strip()))

    f.close()
    return mtx

In [7]:
import math
from queue import PriorityQueue

mtx_neighbor_from_dir = [
    (0, 1), # 0 - East
    (1, 0), # 1 - South
    (0, -1), # 2 - West
    (-1, 0) # 3 - North
]

direction_neighbors = [
    [3, 1], # 0 - East
    [0, 2], # 1 - South
    [1, 3], # 2 - West
    [2, 0] # 3 - North
]

# nodes of the graph: non-wall fields in mtx, multiplied by number of directions
# edges: moving is weight 1, turning is weight 1000

def find_min_cost_path(mtx):
    n = len(mtx)
    m = len(mtx[0])
    start = (n-2, 1, 0)
    end = (1, m-2)
    distances = {}
    for i in range(n):
        for j in range(m):
            if mtx[i][j] != '#':
                for direction in range(4):
                    distances[(i, j, direction)] = math.inf
    distances[start] = 0
    dist_pq = PriorityQueue()
    dist_pq.put((0, start))
    visited = set()

    while not dist_pq.empty():
        current = dist_pq.get()
        while current[1] in visited and not dist_pq.empty():
            current = dist_pq.get()
        if current[1] in visited:
            # all finite distance nodes are visited => no path to end
            return False

        (d, (x, y, direction)) = current
        # if this is the end, we found the shortest path
        if (x, y) == end:
            return d

        # for all neighbors of current, update distance
        # rotate:
        for new_dir in direction_neighbors[direction]:
            if d + 1000 < distances[(x, y, new_dir)]:
                distances[(x, y, new_dir)] = d + 1000
                dist_pq.put((d + 1000, (x, y, new_dir)))
        # move forward:
        move_vect = mtx_neighbor_from_dir[direction]
        x_new, y_new = x + move_vect[0], y + move_vect[1]
        if mtx[x_new][y_new] != '#' and d + 1 < distances[(x_new, y_new, direction)]:
            distances[(x_new, y_new, direction)] = d + 1
            dist_pq.put((d + 1, (x_new, y_new, direction)))
        visited.add((x, y, direction))

In [4]:
def solve1(input_filename):
    mtx = read_input(input_filename)
    return find_min_cost_path(mtx)

## Problem 2

In [6]:
def find_shortest_paths_fied_count(mtx):
    n = len(mtx)
    m = len(mtx[0])
    start = (n-2, 1, 0)
    end = (1, m-2)
    distances = {}
    for i in range(n):
        for j in range(m):
            if mtx[i][j] != '#':
                for direction in range(4):
                    distances[(i, j, direction)] = math.inf
    distances[start] = 0
    shortest_path_origins = { start: [] }
    dist_pq = PriorityQueue()
    dist_pq.put((0, start))
    visited = set()

    min_dist = math.inf
    while not dist_pq.empty():
        current = dist_pq.get()
        while current[1] in visited and not dist_pq.empty():
            current = dist_pq.get()
        if current[1] in visited:
            # all finite distance nodes are visited => we've seen everything in connected component
            break

        (d, (x, y, direction)) = current
        # if this is the end, we found the shortest path
        if (x, y) == end and d < min_dist:
            min_dist = d

        # for all neighbors of current, update distance
        # rotate:
        for new_dir in direction_neighbors[direction]:
            if d + 1000 < distances[(x, y, new_dir)]:
                distances[(x, y, new_dir)] = d + 1000
                shortest_path_origins[(x, y, new_dir)] = [(x, y, direction)]
                dist_pq.put((d + 1000, (x, y, new_dir)))
            elif d + 1000 == distances[(x, y, new_dir)]:
                shortest_path_origins[(x, y, new_dir)].append((x, y, direction))
        # move forward:
        move_vect = mtx_neighbor_from_dir[direction]
        x_new, y_new = x + move_vect[0], y + move_vect[1]
        if mtx[x_new][y_new] != '#':
            if d + 1 < distances[(x_new, y_new, direction)]:
                distances[(x_new, y_new, direction)] = d + 1
                shortest_path_origins[(x_new, y_new, direction)] = [(x, y, direction)]
                dist_pq.put((d + 1, (x_new, y_new, direction)))
            elif d + 1 == distances[(x_new, y_new, direction)]:
                shortest_path_origins[(x_new, y_new, direction)].append((x, y, direction))
        visited.add((x, y, direction))

    fields_in_short_paths = { end }
    nodes_to_check = []
    checked_nodes = set()
    for direction in range(4):
        if distances[(end[0], end[1], direction)] == min_dist:
            nodes_to_check.append((end[0], end[1], direction))
    while len(nodes_to_check) > 0:
        current_node = nodes_to_check.pop(0)
        fields_in_short_paths.add((current_node[0], current_node[1]))
        for node in shortest_path_origins[current_node]:
            if node not in checked_nodes:
                nodes_to_check.append(node)
        checked_nodes.add(current_node)
    return len(fields_in_short_paths)

In [27]:
def solve2(input_filename):
    mtx = read_input(input_filename)
    return find_shortest_paths_fied_count(mtx)